In [6]:
import sys
import os

# Chemin absolu du répertoire racine du projet
project_root = '/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/'

# Ajouter le répertoire src au chemin de recherche des modules
src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)



In [7]:
!python main.py


Traceback (most recent call last):
  File "/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/main.py", line 31, in <module>
    from sqlalchemy.orm import sessionmaker, relationship, declarative_base
ImportError: cannot import name 'declarative_base' from 'sqlalchemy.orm' (/home/conite/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sqlalchemy/orm/__init__.py)


In [10]:
import pandas as pd

df = pd.read_csv('/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/macro_table/macro_table.csv')

In [11]:
df['Topic'] = {}

In [ ]:
from datetime import datetime, timedelta

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    review = row['Reviewer_Text']
    
    try:
        topics = extractor.extract(review, type='SINGLE_SOURCE')
        topics_array = topics['topics']
    
        if len(topics_array) == 0:
            static_topic, static_sentiment, static_sub_topic = generate_static_topics_and_sentiments(row['Reviewer_Sart'])
            new_row = row.copy()
            new_row['Topic'] = static_topic
            new_row['Sentiment'] = static_sentiment
            new_row['Sub_Topic'] = static_sub_topic
            df_macro = pd.concat([df_macro, pd.DataFrame([new_row])], ignore_index=True)
            row_accumulated += 1
        else:
            for tuple_ in topics_array:
                new_row = row.copy()
                new_row['Topic'] = tuple_[0]
                new_row['Sentiment'] = tuple_[1]
                new_row['Sub_Topic'] = tuple_[2]
                df_macro = pd.concat([df_macro, pd.DataFrame([new_row])], ignore_index=True)
                row_accumulated += 1
            
        # Sauvegarder à chaque intervalle défini
        if row_accumulated >= save_interval:
            df_macro.to_csv(output_file, index=False)
            row_accumulated = 0
    except Exception as e:
        print(f"Erreur rencontrée à l'index {index}: {e}")
        pass